## ¿Qué es Dash?

**Dash** es un framework de Python para crear aplicaciones web interactivas y dashboards. Es especialmente útil para:

- Crear dashboards de datos
- Aplicaciones web con visualizaciones interactivas
- Interfaces para análisis de datos
- Aplicaciones de machine learning

## ¿Qué es Plotly?

**Plotly** es una biblioteca de visualización que funciona perfectamente con Dash para crear gráficos interactivos y hermosos.

## Ventajas de usar Dash + Plotly

 **Fácil de aprender** - Solo necesitas Python  
 **Interactividad automática** - Los gráficos son interactivos por defecto  
 **Responsive** - Se adapta a diferentes tamaños de pantalla  
 **Sin JavaScript** - Todo se hace en Python  
 **Integración perfecta** - Dash y Plotly están diseñados para trabajar juntos


## Instalación

Primero necesitas instalar las librerías necesarias:

```bash
pip install dash plotly pandas
```



In [ ]:
!pip install dash plotly pandas

In [5]:
# Importar las librerías necesarias
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

import plotly.io as pio
pio.renderers.default = "notebook" 

## Conceptos Básicos de Dash

### 1. Estructura de una App Dash

Una aplicación Dash tiene 3 componentes principales:

1. **Layout** - Define cómo se ve la aplicación (HTML/CSS)
2. **Callbacks** - Define la interactividad (qué pasa cuando el usuario hace algo)
3. **App** - La aplicación principal que une todo

### 2. Componentes Principales

- `html.Div()` - Contenedor básico (como `<div>` en HTML)
- `dcc.Graph()` - Para mostrar gráficos de Plotly
- `dcc.Dropdown()` - Menú desplegable
- `dcc.Slider()` - Control deslizante
- `html.H1()`, `html.P()` - Títulos y párrafos


## Ejemplo 1: Aplicación Básica

Vamos a crear nuestra primera aplicación Dash simple:


In [ ]:
# Crear datos de ejemplo
np.random.seed(42)
data = {
    'x': np.random.randn(100),
    'y': np.random.randn(100),
    'color': np.random.choice(['A', 'B', 'C'], 100)
}

df = pd.DataFrame(data)

# Crear un gráfico con Plotly
fig = px.scatter(df, x='x', y='y', color='color', 
                 title='Mi primer gráfico interactivo',
                 labels={'x': 'Valor X', 'y': 'Valor Y'})

# Mostrar el gráfico
fig.show()


In [ ]:
# Crear la aplicación Dash
app = dash.Dash(__name__)

# Definir el layout (cómo se ve la aplicación)
app.layout = html.Div([
    html.H1("Mi Primera App con Dash y Plotly", 
            style={'textAlign': 'center', 'color': '#2E86AB'}),
    
    html.P("Esta es una aplicación simple que muestra un gráfico interactivo",
           style={'textAlign': 'center', 'fontSize': 18}),
    
    dcc.Graph(
        id='mi-grafico',
        figure=fig
    ),
    
    html.P("¡Puedes hacer zoom, moverte por el gráfico y hacer hover sobre los puntos!",
           style={'textAlign': 'center', 'fontStyle': 'italic'})
])

# Para ejecutar la aplicación
app.run(debug=True)


## Ejemplo 2: Aplicación con Interactividad

Ahora vamos a crear una aplicación más avanzada con controles interactivos:


In [ ]:
# Crear datos más complejos para el ejemplo
np.random.seed(123)
years = list(range(2010, 2024))
countries = ['España', 'Francia', 'Alemania', 'Italia', 'Portugal']

# Generar datos de ventas por país y año
ventas_data = []
for country in countries:
    for year in years:
        ventas = np.random.randint(1000, 5000)
        ventas_data.append({
            'País': country,
            'Año': year,
            'Ventas': ventas
        })

df_ventas = pd.DataFrame(ventas_data)
print("Datos de ventas creados:")
print(df_ventas.head())


In [ ]:
# Crear una nueva aplicación Dash más avanzada
app2 = dash.Dash(__name__)

# Layout con controles interactivos
app2.layout = html.Div([
    html.H1("Dashboard de Ventas Interactivo", 
            style={'textAlign': 'center', 'color': '#2E86AB', 'marginBottom': 30}),
    
    # Controles en una fila
    html.Div([
        html.Div([
            html.Label("Selecciona un país:"),
            dcc.Dropdown(
                id='pais-dropdown',
                options=[{'label': pais, 'value': pais} for pais in countries],
                value='España',
                style={'width': '100%'}
            )
        ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '5%'}),
        
        html.Div([
            html.Label("Selecciona el rango de años:"),
            dcc.RangeSlider(
                id='año-slider',
                min=2010,
                max=2023,
                value=[2015, 2020],
                marks={str(year): str(year) for year in range(2010, 2024, 2)},
                step=1
            )
        ], style={'width': '65%', 'display': 'inline-block'})
    ], style={'marginBottom': 30}),
    
    # Gráfico principal
    dcc.Graph(id='ventas-grafico'),
    
    # Gráfico secundario
    dcc.Graph(id='comparacion-grafico')
])


### Callbacks - El Corazón de la Interactividad

Los **callbacks** son funciones que se ejecutan cuando el usuario interactúa con los controles. Definen qué pasa cuando cambias un dropdown, slider, etc.


In [ ]:
# Callback para el gráfico principal
@app2.callback(
    Output('ventas-grafico', 'figure'),
    [Input('pais-dropdown', 'value'),
     Input('año-slider', 'value')]
)
def actualizar_grafico_ventas(pais_seleccionado, años_seleccionados):
    # Filtrar datos según las selecciones
    df_filtrado = df_ventas[
        (df_ventas['País'] == pais_seleccionado) & 
        (df_ventas['Año'] >= años_seleccionados[0]) & 
        (df_ventas['Año'] <= años_seleccionados[1])
    ]
    
    # Crear el gráfico
    fig = px.line(df_filtrado, x='Año', y='Ventas', 
                  title=f'Ventas de {pais_seleccionado} ({años_seleccionados[0]}-{años_seleccionados[1]})',
                  markers=True)
    
    # Personalizar el gráfico
    fig.update_layout(
        plot_bgcolor='white',
        xaxis_title="Año",
        yaxis_title="Ventas (unidades)",
        height=400
    )
    
    return fig

# Callback para el gráfico de comparación
@app2.callback(
    Output('comparacion-grafico', 'figure'),
    [Input('año-slider', 'value')]
)
def actualizar_grafico_comparacion(años_seleccionados):
    # Filtrar datos por años
    df_filtrado = df_ventas[
        (df_ventas['Año'] >= años_seleccionados[0]) & 
        (df_ventas['Año'] <= años_seleccionados[1])
    ]
    
    # Calcular promedio por país
    df_promedio = df_filtrado.groupby('País')['Ventas'].mean().reset_index()
    
    # Crear gráfico de barras
    fig = px.bar(df_promedio, x='País', y='Ventas',
                 title=f'Promedio de Ventas por País ({años_seleccionados[0]}-{años_seleccionados[1]})',
                 color='Ventas',
                 color_continuous_scale='Blues')
    
    fig.update_layout(
        plot_bgcolor='white',
        height=400,
        showlegend=False
    )
    
    return fig




## Tipos de Gráficos Comunes con Plotly

Plotly ofrece muchos tipos de gráficos. Aquí tienes los más útiles:


In [ ]:
# Crear datos de ejemplo para mostrar diferentes tipos de gráficos
datos_ejemplo = pd.DataFrame({
    'categoria': ['A', 'B', 'C', 'D', 'E'],
    'valores': [23, 45, 56, 78, 32],
    'valores2': [34, 25, 67, 89, 45],
    'fechas': pd.date_range('2023-01-01', periods=5, freq='M')
})

print("Datos de ejemplo:")
print(datos_ejemplo)


In [ ]:
# 1. Gráfico de Barras
fig_barras = px.bar(datos_ejemplo, x='categoria', y='valores',
                   title='Gráfico de Barras',
                   color='valores',
                   color_continuous_scale='viridis')
fig_barras.show()


In [ ]:
# 2. Gráfico de Líneas
fig_lineas = px.line(datos_ejemplo, x='fechas', y='valores',
                    title='Gráfico de Líneas',
                    markers=True)
fig_lineas.show()


In [ ]:
# 3. Gráfico de Dispersión (Scatter)
fig_scatter = px.scatter(datos_ejemplo, x='valores', y='valores2',
                        title='Gráfico de Dispersión',
                        color='categoria',
                        size='valores',
                        hover_data=['fechas'])
fig_scatter.show()


In [ ]:
# 4. Gráfico de Pie (Circular)
fig_pie = px.pie(datos_ejemplo, values='valores', names='categoria',
                 title='Gráfico Circular')
fig_pie.show()


In [ ]:
# 5. Histograma
np.random.seed(42)
datos_histograma = np.random.normal(100, 15, 1000)

fig_histograma = px.histogram(x=datos_histograma, nbins=30,
                             title='Histograma',
                             labels={'x': 'Valores', 'y': 'Frecuencia'})
fig_histograma.show()


## Ejemplo 3: Dashboard Completo con Múltiples Gráficos

Vamos a crear un dashboard más completo que combine varios tipos de gráficos:


In [ ]:
# Crear datos más complejos para el dashboard
np.random.seed(456)
departamentos = ['Ventas', 'Marketing', 'IT', 'RRHH', 'Finanzas']
empleados = ['Ana', 'Carlos', 'María', 'Luis', 'Elena', 'Pedro', 'Sofia', 'Miguel']

# Generar datos de rendimiento
datos_rendimiento = []
for _ in range(100):
    datos_rendimiento.append({
        'Empleado': np.random.choice(empleados),
        'Departamento': np.random.choice(departamentos),
        'Puntuacion': np.random.randint(1, 11),
        'Horas_Trabajadas': np.random.randint(30, 50),
        'Proyectos_Completados': np.random.randint(1, 8)
    })

df_rendimiento = pd.DataFrame(datos_rendimiento)
print("Datos de rendimiento creados:")
print(df_rendimiento.head())


In [ ]:
# Crear el dashboard completo
app3 = dash.Dash(__name__)

app3.layout = html.Div([
    html.H1("Dashboard de Rendimiento de Empleados", 
            style={'textAlign': 'center', 'color': '#2E86AB', 'marginBottom': 30}),
    
    # Controles
    html.Div([
        html.Div([
            html.Label("Selecciona un departamento:"),
            dcc.Dropdown(
                id='depto-dropdown',
                options=[{'label': depto, 'value': depto} for depto in departamentos],
                value='Ventas',
                style={'width': '100%'}
            )
        ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '5%'}),
        
        html.Div([
            html.Label("Tipo de gráfico:"),
            dcc.Dropdown(
                id='tipo-grafico',
                options=[
                    {'label': 'Barras', 'value': 'bar'},
                    {'label': 'Líneas', 'value': 'line'},
                    {'label': 'Dispersión', 'value': 'scatter'}
                ],
                value='bar',
                style={'width': '100%'}
            )
        ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '5%'}),
        
        html.Div([
            html.Label("Métrica:"),
            dcc.Dropdown(
                id='metrica-dropdown',
                options=[
                    {'label': 'Puntuación', 'value': 'Puntuacion'},
                    {'label': 'Horas Trabajadas', 'value': 'Horas_Trabajadas'},
                    {'label': 'Proyectos Completados', 'value': 'Proyectos_Completados'}
                ],
                value='Puntuacion',
                style={'width': '100%'}
            )
        ], style={'width': '30%', 'display': 'inline-block'})
    ], style={'marginBottom': 30}),
    
    # Gráficos en dos columnas
    html.Div([
        html.Div([
            dcc.Graph(id='grafico-principal')
        ], style={'width': '50%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.Graph(id='grafico-secundario')
        ], style={'width': '50%', 'display': 'inline-block'})
    ]),
    
    # Gráfico de abajo
    html.Div([
        dcc.Graph(id='grafico-distribucion')
    ], style={'marginTop': 30})
])


In [ ]:
# Callbacks para el dashboard completo
@app3.callback(
    Output('grafico-principal', 'figure'),
    [Input('depto-dropdown', 'value'),
     Input('tipo-grafico', 'value'),
     Input('metrica-dropdown', 'value')]
)
def actualizar_grafico_principal(depto, tipo_grafico, metrica):
    # Filtrar datos por departamento
    df_filtrado = df_rendimiento[df_rendimiento['Departamento'] == depto]
    
    # Agrupar por empleado y calcular promedio
    df_agrupado = df_filtrado.groupby('Empleado')[metrica].mean().reset_index()
    
    # Crear gráfico según el tipo seleccionado
    if tipo_grafico == 'bar':
        fig = px.bar(df_agrupado, x='Empleado', y=metrica,
                     title=f'{metrica} por Empleado - {depto}',
                     color=metrica,
                     color_continuous_scale='viridis')
    elif tipo_grafico == 'line':
        fig = px.line(df_agrupado, x='Empleado', y=metrica,
                      title=f'{metrica} por Empleado - {depto}',
                      markers=True)
    else:  # scatter
        fig = px.scatter(df_agrupado, x='Empleado', y=metrica,
                        title=f'{metrica} por Empleado - {depto}',
                        size=metrica,
                        color=metrica,
                        color_continuous_scale='plasma')
    
    fig.update_layout(height=400, plot_bgcolor='white')
    return fig

@app3.callback(
    Output('grafico-secundario', 'figure'),
    [Input('depto-dropdown', 'value')]
)
def actualizar_grafico_secundario(depto):
    # Comparación de métricas por departamento
    df_deptos = df_rendimiento.groupby('Departamento').agg({
        'Puntuacion': 'mean',
        'Horas_Trabajadas': 'mean',
        'Proyectos_Completados': 'mean'
    }).reset_index()
    
    # Crear gráfico de barras apiladas
    fig = go.Figure()
    
    fig.add_trace(go.Bar(name='Puntuación', x=df_deptos['Departamento'], 
                        y=df_deptos['Puntuacion']))
    fig.add_trace(go.Bar(name='Horas', x=df_deptos['Departamento'], 
                        y=df_deptos['Horas_Trabajadas']))
    fig.add_trace(go.Bar(name='Proyectos', x=df_deptos['Departamento'], 
                        y=df_deptos['Proyectos_Completados']))
    
    fig.update_layout(title='Comparación de Métricas por Departamento',
                     barmode='group', height=400, plot_bgcolor='white')
    
    return fig

@app3.callback(
    Output('grafico-distribucion', 'figure'),
    [Input('depto-dropdown', 'value')]
)
def actualizar_grafico_distribucion(depto):
    # Histograma de distribución de puntuaciones
    df_filtrado = df_rendimiento[df_rendimiento['Departamento'] == depto]
    
    fig = px.histogram(df_filtrado, x='Puntuacion', nbins=10,
                      title=f'Distribución de Puntuaciones - {depto}',
                      color_discrete_sequence=['#2E86AB'])
    
    fig.update_layout(height=300, plot_bgcolor='white')
    return fig


## Cómo Ejecutar las Aplicaciones

Para ejecutar cualquiera de las aplicaciones que hemos creado, simplemente ejecuta:

```python
# Para la aplicación básica
app.run_server(debug=True)

# Para la aplicación interactiva
app2.run_server(debug=True)

# Para el dashboard completo
app3.run_server(debug=True)
```

**Nota:** En un notebook de Jupyter, es mejor usar:
```python
app.run_server(debug=True, port=8050)
```

Luego abre tu navegador en `http://localhost:8050`


## Consejos y Mejores Prácticas

### 1. Organización del Código
- Separa el layout de los callbacks
- Usa funciones para generar gráficos
- Mantén los datos en variables separadas

### 2. Performance
- Usa `dcc.Store` para datos grandes
- Implementa debouncing en callbacks
- Optimiza las consultas a la base de datos

### 3. Diseño
- Usa CSS personalizado para mejorar la apariencia
- Mantén consistencia en colores y fuentes
- Haz que sea responsive (se adapte a móviles)

### 4. Debugging
- Usa `debug=True` durante desarrollo
- Imprime valores en callbacks para debuggear
- Usa `dash.callback_context` para información del callback


## Ejemplo Final: Aplicación con Estilos CSS

Vamos a crear una aplicación con estilos personalizados:


In [ ]:
# Aplicación final con estilos CSS
app_final = dash.Dash(__name__)

# Estilos CSS personalizados
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Datos de ejemplo para la aplicación final
datos_ventas = pd.DataFrame({
    'Mes': ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun'],
    'Ventas': [1000, 1200, 1100, 1400, 1300, 1600],
    'Gastos': [800, 900, 850, 1000, 950, 1100]
})

# Layout con estilos
app_final.layout = html.Div([
    # Header con estilo
    html.Div([
        html.H1("Dashboard de Ventas 2024", 
                style={'textAlign': 'center', 'color': 'white', 'margin': 0}),
        html.P("Análisis de rendimiento mensual", 
               style={'textAlign': 'center', 'color': 'white', 'margin': '10px 0'})
    ], style={'backgroundColor': '#2E86AB', 'padding': '20px', 'marginBottom': '30px'}),
    
    # Controles con estilo
    html.Div([
        html.Div([
            html.Label("Selecciona el tipo de visualización:", 
                      style={'fontWeight': 'bold', 'marginBottom': '10px'}),
            dcc.Dropdown(
                id='tipo-visualizacion',
                options=[
                    {'label': 'Gráfico de Líneas', 'value': 'line'},
                    {'label': 'Gráfico de Barras', 'value': 'bar'},
                    {'label': 'Área', 'value': 'area'}
                ],
                value='line',
                style={'width': '100%'}
            )
        ], style={'width': '48%', 'display': 'inline-block', 'marginRight': '4%'}),
        
        html.Div([
            html.Label("Mostrar:", 
                      style={'fontWeight': 'bold', 'marginBottom': '10px'}),
            dcc.Checklist(
                id='datos-checklist',
                options=[
                    {'label': 'Ventas', 'value': 'Ventas'},
                    {'label': 'Gastos', 'value': 'Gastos'}
                ],
                value=['Ventas'],
                style={'marginTop': '10px'}
            )
        ], style={'width': '48%', 'display': 'inline-block'})
    ], style={'backgroundColor': '#f8f9fa', 'padding': '20px', 'borderRadius': '5px', 'marginBottom': '30px'}),
    
    # Gráfico principal
    html.Div([
        dcc.Graph(id='grafico-principal-final')
    ], style={'marginBottom': '30px'}),
    
    # Métricas en tarjetas
    html.Div([
        html.Div([
            html.H3(id='total-ventas', style={'textAlign': 'center', 'color': '#2E86AB'}),
            html.P("Total Ventas", style={'textAlign': 'center', 'margin': 0})
        ], style={'width': '30%', 'display': 'inline-block', 'backgroundColor': '#e8f4f8', 
                 'padding': '20px', 'margin': '1%', 'borderRadius': '5px', 'textAlign': 'center'}),
        
        html.Div([
            html.H3(id='total-gastos', style={'textAlign': 'center', 'color': '#e74c3c'}),
            html.P("Total Gastos", style={'textAlign': 'center', 'margin': 0})
        ], style={'width': '30%', 'display': 'inline-block', 'backgroundColor': '#fdeaea', 
                 'padding': '20px', 'margin': '1%', 'borderRadius': '5px', 'textAlign': 'center'}),
        
        html.Div([
            html.H3(id='beneficio', style={'textAlign': 'center', 'color': '#27ae60'}),
            html.P("Beneficio", style={'textAlign': 'center', 'margin': 0})
        ], style={'width': '30%', 'display': 'inline-block', 'backgroundColor': '#eafaf1', 
                 'padding': '20px', 'margin': '1%', 'borderRadius': '5px', 'textAlign': 'center'})
    ], style={'marginBottom': '30px'})
])


In [ ]:
# Callbacks para la aplicación final
@app_final.callback(
    Output('grafico-principal-final', 'figure'),
    [Input('tipo-visualizacion', 'value'),
     Input('datos-checklist', 'value')]
)
def actualizar_grafico_final(tipo, datos_seleccionados):
    fig = go.Figure()
    
    colores = {'Ventas': '#2E86AB', 'Gastos': '#e74c3c'}
    
    for dato in datos_seleccionados:
        if tipo == 'line':
            fig.add_trace(go.Scatter(x=datos_ventas['Mes'], y=datos_ventas[dato],
                                   mode='lines+markers', name=dato,
                                   line=dict(color=colores[dato], width=3),
                                   marker=dict(size=8)))
        elif tipo == 'bar':
            fig.add_trace(go.Bar(x=datos_ventas['Mes'], y=datos_ventas[dato],
                               name=dato, marker_color=colores[dato]))
        else:  # area
            fig.add_trace(go.Scatter(x=datos_ventas['Mes'], y=datos_ventas[dato],
                                   mode='lines', fill='tonexty', name=dato,
                                   line=dict(color=colores[dato])))
    
    fig.update_layout(
        title='Análisis de Ventas y Gastos 2024',
        plot_bgcolor='white',
        height=400,
        showlegend=True,
        legend=dict(x=0.02, y=0.98)
    )
    
    return fig

@app_final.callback(
    [Output('total-ventas', 'children'),
     Output('total-gastos', 'children'),
     Output('beneficio', 'children')],
    [Input('datos-checklist', 'value')]
)
def actualizar_metricas(datos_seleccionados):
    total_ventas = datos_ventas['Ventas'].sum()
    total_gastos = datos_ventas['Gastos'].sum()
    beneficio = total_ventas - total_gastos
    
    return f"${total_ventas:,}", f"${total_gastos:,}", f"${beneficio:,}"

print("✅ Aplicación final con estilos creada")
